In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv('diabetes.csv')
df=df.drop(columns='family_histroy')
df.replace(4.860753, np.nan, inplace=True)
df.dropna(inplace=True)
df["smoking"] = df["smoking"].replace({1.0: 0, 2.0: 0, 3.0: 1})
df_copy=df
df_copy["smoking"] = df["smoking"].astype(int)
X = df_copy.drop('smoking', axis=1)
y = df_copy['smoking']

In [3]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb



skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
   
    n_estimators = trial.suggest_int('n_estimators', 100, 300, step=20)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)

    scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

       
        #scaler = MinMaxScaler()
        scaler =StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

       
        model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_jobs=-1,
            random_state=42
        )

        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        acc = accuracy_score(y_val, y_pred)
        scores.append(acc)

    return np.mean(scores)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  

print("best_params:", study.best_params)
print(f"best_value: {study.best_value:.4f}")

[I 2025-08-22 16:21:37,626] A new study created in memory with name: no-name-44a74835-1520-4c3c-b74d-57bfe748a3de
[I 2025-08-22 16:21:38,642] Trial 0 finished with value: 0.761963190184049 and parameters: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.021720891163567103}. Best is trial 0 with value: 0.761963190184049.
[I 2025-08-22 16:21:39,015] Trial 1 finished with value: 0.7668711656441717 and parameters: {'n_estimators': 180, 'max_depth': 5, 'learning_rate': 0.033453956330989076}. Best is trial 1 with value: 0.7668711656441717.
[I 2025-08-22 16:21:39,893] Trial 2 finished with value: 0.7585889570552147 and parameters: {'n_estimators': 240, 'max_depth': 8, 'learning_rate': 0.06816259187683438}. Best is trial 1 with value: 0.7668711656441717.
[I 2025-08-22 16:21:40,511] Trial 3 finished with value: 0.7610429447852761 and parameters: {'n_estimators': 220, 'max_depth': 6, 'learning_rate': 0.013175991915539302}. Best is trial 1 with value: 0.7668711656441717.
[I 2025-08-22 16:

best_params: {'n_estimators': 120, 'max_depth': 4, 'learning_rate': 0.0421540417317611}
best_value: 0.7712
